In [18]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import os

#read data and make pd dataframe
insurance_data = pd.read_csv("./Insurance_Data.csv")
insurance_data.head()
data_frame = pd.DataFrame(insurance_data)



# Removing non categorical features and status for now 
cols = [1,2,3,10,11]
data_frame.drop(data_frame.columns[cols], axis=1, inplace=True)

#count the amount of classes for each categorical feature
class_count = {}
for feature in data_frame:
    classes = data_frame[feature].value_counts()
    class_count[feature] = classes

print(class_count)

{'Company': Company
Anthem Health Plans, Inc                                  6696
ConnectiCare Benefits, Inc.                               1980
UnitedHealthcare Insurance Company                        1551
ConnectiCare Insurance Company, Inc                       1227
Cigna Health and Life Insurance Company                   1191
                                                          ... 
Maxum Indemnity Company                                      1
Ironshore Indemnity Inc.                                     1
Century Surety Company                                       1
GuideOne National Insurance Company                          1
Samsung Fire & Marine Insurance Co., Ltd (U.S. Branch)       1
Name: count, Length: 765, dtype: int64, 'Coverage': Coverage
Individual Private Passenger             9663
Group                                    6069
A & H                                    5813
Individual                               5039
Homeowners                               4